In [92]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import random
import io
import re

In [93]:
url_list = [
    'https://www.seloger.com/annonces/achat/appartement/paris-19eme-75/pont-de-flandre/226482845.htm',
    'https://www.seloger.com/annonces/achat-de-prestige/appartement/paris-16eme-75/chaillot/231397633.htm',
    'https://www.seloger.com/annonces/achat/appartement/paris-18eme-75/montmartre/237248723.htm',
    'https://www.seloger.com/annonces/achat/appartement/paris-18eme-75/amiraux-simplon-poissonniers/236600843.htm',
    'https://www.seloger.com/annonces/achat/appartement/paris-12eme-75/bel-air-nord/237070323.htm',
    'https://www.seloger.com/annonces/achat/appartement/paris-16eme-75/porte-dauphine/237216193.htm',
    'https://www.seloger.com/annonces/achat/appartement/paris-18eme-75/moskowa-porte-montmartre-porte-de-clignancourt/234560645.htm'
]

dataset = pd.read_csv('dataset.csv', index_col='index')
for url in url_list:
    # REQUEST
    response = requests.get(url,headers= {'User-Agent': 'Safari/537.36'},timeout= random.randint(7, 14))
    soup = BeautifulSoup(response.text,'html.parser')
    data = {}

    # DATE
    data['date'] = pd.Timestamp.now()

    # REF SELOGER
    ref_code = re.search(r'\d{8}', url).group()

    # TYPE DE LOGEMENT
    s = soup.find('div',{"class":"Summarystyled__Title-sc-1u9xobv-4 dbveQQ"}).text
    if "Appartement" in s: 
        data['type'] = 'apartment'
    elif "Maison" in s:
        data['type'] = 'house'
    else:
        data['type'] = 'other'

    # CODE POSTAL
    s = soup.find('span', {'class' : "Localizationstyled__City-sc-gdkcr2-1 bgtLnh"}).text
    data['zip_code'] = re.search(r'\((\d{5})\)', s).group(1)

    # PRIX AFFICHE
    s = soup.find('span', {'class': "global-styles__TextNoWrap-sc-1gbe8ip-6 gxurQr"}).text
    s = s.replace(" ", "").replace("€", "").strip()
    data['price'] = s

    # PIECE / CHAMBRE / SURFACE / ETAGE
    s_list = [s.text for s in soup.find_all('div',{'class':"Tags__TagContainer-sc-edpl7u-0 EPxew"})]
    data['room'] = s_list[0].split()[0]
    data['bed'] = s_list[1].split()[0]
    data['area'] = s_list[2].split()[0]
    floor = s_list[3].split()[-1].split('/')
    data['floor'] = floor[0]
    data['max_floor'] = floor[1]

    # DPE
    try:
        s = soup.find('div', {'class' : "Previewstyled__Grade-sc-k3u73o-6 ehFYCZ"}).text
        data['dpe'] = s
    except:
        data['dpe'] = np.nan

    dataset = dataset.assign(**{ref_code: pd.Series(data)})
dataset.to_csv('dataset.csv')


In [94]:
dataset

,22648284,23139763,23724872,23660084,23707032,23721619,23456064
index,,,,,,,
date,2025-03-19 11:29:29.264884,2025-03-19 11:29:30.382178,2025-03-19 11:29:31.828973,2025-03-19 11:29:33.791659,2025-03-19 11:29:34.858424,2025-03-19 11:29:35.263384,2025-03-19 11:29:35.677436
type,other,apartment,apartment,apartment,apartment,apartment,apartment
price,620000,1445000,335000,199000,780000,630000,599000
zip_code,75019,75016,75018,75018,75012,75016,75018
room,4,4,2,2,4,2,4
bed,3,2,1,1,2,1,3
area,110,106,31,28,91,55,81
floor,0,1,1,5,1,0,3
max_floor,12,6,5,5,9,5,4
